# Caso Práctico (tips) - ENGIE
*Jose Enrique Zafra Mena*

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import json 

## Carga de los datos

In [52]:
facturas_df = pd.read_excel('datos/datos-tips/AGIkey_facturas.xlsx')
mapping_df = pd.read_excel('datos/Mapping.xlsx')
IAC_df = pd.read_excel('datos/datos-tips/MEDG2_IAC_GETRA_ABRIL_2024_06_18_13_34.xlsx')
CTV_df = pd.read_excel('datos/datos-tips/JVLNG_CTV_STOK_ABRIL_2024_06_18_13_34.xlsx')

## Primera parte

### Filtrado de datos relevantes

#### Facturas
Solo nos interesan ciertas columnas: Factura, Importe, Fecha Factura.

Además, cambiamos el formato de la fecha para poder filtrar por Abril.

Salen *warnings*, pero no tienen relevancia.

In [16]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen','ServicioFacturado']]

# solo queremos algunos valores de Origen: C.I.Almería y Entrada PVB
facturas_filtrado_df = facturas_df[(facturas_df['Origen'].isin(['C.I. Almería'])) & (facturas_df['ServicioFacturado'].isin(['Entrada PVB']))]

# cambiamos el formato del importe
facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',', '.').astype(float)

C:\Users\kikek\AppData\Local\Temp\ipykernel_3688\3425517899.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',', '.').astype(float)


In [31]:
# cambiamos el formato de la fecha
facturas_filtrado_df['FechaFactura'] = pd.to_datetime(facturas_filtrado_df['FechaFactura'], utc = True)

facturas_filtrado_df.head(10)

# filtramos para abril del 24
facturas_filtrado_df = facturas_filtrado_df[(facturas_filtrado_df['FechaFactura'].dt.month == 4) & (facturas_filtrado_df['FechaFactura'].dt.year == 2024)]

facturas_filtrado_df.head(10)

,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
570,2324005598,536445.62,2024-04-04 00:00:00+00:00,C.I. Almería,Entrada PVB


#### Mapping

In [22]:
mapping_df = mapping_df.drop_duplicates()
print(mapping_df.shape)

# solo nos interesan algunos valores: Commodity == IAC
mapping_filtrado_df = mapping_df[(mapping_df['Commodity'].isin(['IAC']))]
mapping_filtrado_df.head(200)

(29, 5)


,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


#### Deals 
Solo queremos las columnas: Deal, Cantidad, Start-DeliveryDate, End-DeliveryDate, Procedencia (SUGST_AOC_STOK), Subflow 1, Date 1, Subcantidad 1, Subflow 2, Date 2, Subcantidad 2.

En la columna flows vemos que hay datos en json. Tratamos esa columna para sacar los datos a un formato mejor. También mantemos el Id del deal para cada subflow.


In [23]:
IAC_df = IAC_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
IAC_filtrado_df = IAC_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
IAC_filtrado_df = IAC_filtrado_df.rename(columns={'Id':'Id Deal'})

IAC_filtrado_df.head()

,Id Deal,Flows,StartDeliveryDate,EndDeliveryDate
0,37591650,"[{'Currency': 'EUR', 'Amount': 105279.586144, ...",2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00
1,37056084,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 9219...",2023-01-01T00:00:00+01:00,2023-12-31T00:00:00+01:00


In [24]:
# definimos una función para tratar los json (la tendremos que usar varias veces)
def clean_json(string):
    if isinstance(string, str):
        string = string.replace("'", '"')
        string = string.replace("None", 'null')
        string = string.replace("True", 'true')
        string = string.replace("False", 'false')
    return string

# la usamos sobre la columna de flows
flows_cleaned = IAC_filtrado_df['Flows'].apply(clean_json)


# Ahora creamos otra función para extraer los datos de json a diccionarios de python
def extract_json(flows, id_deal):
    if not flows or not isinstance(flows, list):
        return pd.DataFrame()
    data = {
        'Id Deal': [],
        'Currency': [],
        'Amount': [],
        'Id': [],
        'Date': [],
        'FlowType': [],
        'PayReceive': []
    }
    for item in flows:
        if isinstance(item, dict):
            data['Id Deal'].append(id_deal)
            for key in data.keys():
                if key != 'Id Deal':
                    data[key].append(item.get(key))
        else:
            print(f"Unexpected type {type(item)}")
    return pd.DataFrame(data)

# creamos un df para meter los datos
datos_json = []

for idx, row in IAC_filtrado_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        print(f"Procesamos fila {idx} con Id Deal {row['Id Deal']}")
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos_json_final = pd.concat(datos_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos_json_final = datos_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos_json_final = datos_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos_json_final.head(100)

Procesamos fila 0 con Id Deal 37591650
Procesamos fila 1 con Id Deal 37056084


,Id Deal,Id Flow,Importe Flow,Fecha Flow,Subflow
0,37591650,93540225,105279.586144,2024-02-20T00:00:00+01:00,1
1,37591650,93540224,409582.490000,2024-02-20T00:00:00+01:00,2
2,37591650,93540227,120318.957648,2024-03-20T00:00:00+01:00,1
3,37591650,93540226,383444.510000,2024-03-20T00:00:00+01:00,2
4,37591650,93540229,128084.859614,2024-04-22T00:00:00+02:00,1
5,37591650,93540228,408200.220000,2024-04-22T00:00:00+02:00,2
6,37591650,93540231,130004.564430,2024-05-20T00:00:00+02:00,1
7,37591650,93540230,395032.480000,2024-05-20T00:00:00+02:00,2
8,37591650,93540233,134338.049911,2024-06-20T00:00:00+02:00,1
9,37591650,93540232,408200.220000,2024-06-20T00:00:00+02:00,2


Lo anterior solo es la columna que teníamos de flows. Ahora lo unimos con el resto del dataframe original (deals IAC). Tan solo tenemos que añadirle las columnas de las fechas iniciales y finales, según el valor de Id Deal que tenga.

También cambiamos el formato de las fechas y filtramos para el mes de Abril

In [38]:
IAC_final_df = (pd.merge(IAC_filtrado_df, datos_json_final, on='Id Deal', how='inner')).drop_duplicates()
IAC_final_df = IAC_final_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor IAC para que sea más fácil el merge
IAC_final_df['Commodity']='IAC'

# cambiamos el formato de las fechas (por lo menos el de Fecha Flow)
IAC_final_df['Fecha Flow'] = pd.to_datetime(IAC_final_df['Fecha Flow'], utc=True)

# filtramos para el mes de Abril
IAC_final_df = IAC_final_df[IAC_final_df['Fecha Flow'].dt.month == 4]
IAC_final_df = IAC_final_df[IAC_final_df['Fecha Flow'].dt.year == 2024]

IAC_final_df.head(100)


,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity
4,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-21 22:00:00+00:00,1,IAC
5,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-21 22:00:00+00:00,2,IAC


### Merge de Mapping con Deals
Hacemos el merge con Commodity

In [39]:
IAC_merged_df = pd.merge(IAC_final_df, mapping_filtrado_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
IAC_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)


IAC_merged_df.head(50)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity,Origen,Servicio facturado
0,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-21 22:00:00+00:00,1,IAC,C.I. Almería,Entrada PVB
1,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-21 22:00:00+00:00,2,IAC,C.I. Almería,Entrada PVB


### Merge de Mapping con Facturas
En este caso nos vale bien con Origen, o bien con Servicio Facturado.

In [40]:
facturas_merged_df = pd.merge(facturas_filtrado_df, mapping_filtrado_df, on='Origen', how='inner')


facturas_merged_df = facturas_merged_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged_df = facturas_merged_df.loc[:, ~facturas_merged_df.columns.duplicated()]

facturas_merged_df.head(100)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2324005598,536445.62,2024-04-04 00:00:00+00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


### Conciliación
**Se hace según *Origen* + *Servicio Facturado***

Mapping y Facturas comparten *Origen* y *Servicio Facturado*

Vemos la cantidad según Facturas, la cantidad según Deals, y hacemos la diferencia

#### Comparación entre Facturas y Deals


In [47]:
# agrupamos y sumamos importes
IAC_grouped = IAC_merged_df.groupby(['Id Deal','Fecha Flow','Origen', 'Servicio facturado', 'Subflow'])['Importe Flow'].sum()

IAC_grouped.head(100)

Id Deal   Fecha Flow                 Origen        Servicio facturado  Subflow
37591650  2024-04-21 22:00:00+00:00  C.I. Almería  Entrada PVB         1          128084.859614
                                                                       2          408200.220000
Name: Importe Flow, dtype: float64

In [51]:
# hacemos lo mismo para facturas
facturas_grouped = facturas_merged_df.groupby(['NumeroFactura','FechaFactura','Origen', 'Servicio facturado'])['Importe'].sum()

facturas_grouped.head(100)

NumeroFactura  FechaFactura               Origen        Servicio facturado
2324005598     2024-04-04 00:00:00+00:00  C.I. Almería  Entrada PVB           536445.62
Name: Importe, dtype: float64

#### DataFrame de conciliación
Falta poner columnas de Subflows, Numero de Factura, Id Deal

In [46]:
# creamos un dataframe solo con el importe total de los deals
IAC_total_df = IAC_merged_df.groupby(['Id Deal','Origen', 'Servicio facturado'])['Importe Flow'].sum()

# unimos los dataframes para comparar
conciliacion_df = pd.merge(IAC_total_df, facturas_grouped, on=['Origen', 'Servicio facturado'], how='outer')

# hallamos la diferencia 
conciliacion_df['Factura-Deal'] = conciliacion_df['Importe'] - conciliacion_df['Importe Flow']  

conciliacion_df.head()

,,Importe Flow,Importe,Factura-Deal
Origen,Servicio facturado,,,
C.I. Almería,Entrada PVB,536285.079614,536445.62,160.540386


## Segunda Parte
Seguiremos el mismo procedimiento que antes.

### Filtrado de datos relevantes

#### Facturas
Solo queremos Origen == TVB y Servicio Facturado == Almacenamiento TVB.

Filtrar para mes de marzo, fecha factura Abril 2024

In [55]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen', 'ServicioFacturado']]

# filtramos los valores que queremos
facturas_filtrado2_df = facturas_df[(facturas_df['Origen'].isin(['TVB'])) & (facturas_df['ServicioFacturado'].isin(['Almacenamiento TVB']))]

# cambiamos el formato del importe
facturas_filtrado2_df['Importe'] = facturas_filtrado2_df['Importe'].str.replace(',', '.').astype(float)

# cambiamos el formato de la fecha
facturas_filtrado2_df['FechaFactura'] = pd.to_datetime(facturas_filtrado2_df['FechaFactura'], utc=True)

# filtramos para abril del 24
facturas_filtrado2_df = facturas_filtrado2_df[(facturas_filtrado2_df['FechaFactura'].dt.month == 4) & (facturas_filtrado2_df['FechaFactura'].dt.year == 2024)]

facturas_filtrado2_df.head(100)

C:\Users\kikek\AppData\Local\Temp\ipykernel_3688\2108121968.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado2_df['Importe'] = facturas_filtrado2_df['Importe'].str.replace(',', '.').astype(float)
C:\Users\kikek\AppData\Local\Temp\ipykernel_3688\2108121968.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado2_df['FechaFactura'] = pd.to_datetime(facturas_filtrado2_df['FechaFactura'], utc=True)


,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
279,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
280,2024164583,718.60,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
281,2024164584,1015.95,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
282,2024164585,1017.59,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
283,2024164586,1030.97,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
284,2024164587,1041.67,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
285,2024164588,1965.66,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
286,2024164589,1472.13,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
287,2024164590,1930.23,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
288,2024164591,1787.43,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB


#### Mapping
Ahora tenemos que filtrar para Commodity == CTV, y Portfolio == JVLNG.

Realmente creo que esto se podría optimizar solo con el paso del merge, pero ahora no se me ocurre cómo.

In [62]:
mapping_df = mapping_df.drop_duplicates()
mapping_filtrado2_df = mapping_df[(mapping_df['Commodity'].isin(['CTV'])) & (mapping_df['Portfolio'].isin(['JVLNG']))]

mapping_filtrado2_df.head(100)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK


#### Deals
Se supone que ya está filtrado en el deal (la fecha??). Hay que localizar los subflows.

### Merge de Mapping con Deals

### Merge de Mapping con Facturas

In [58]:
facturas_merged2_df = pd.merge(facturas_filtrado2_df, mapping_df, on='Origen', how='inner')

facturas_merged2_df = facturas_merged2_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged2_df = facturas_merged2_df.loc[:, ~facturas_merged2_df.columns.duplicated()]

facturas_merged2_df.head(100)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
1,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
2,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
3,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
4,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
...,...,...,...,...,...,...,...,...
95,2024164585,1017.59,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2F,GETRA
96,2024164586,1030.97,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
97,2024164586,1030.97,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
98,2024164586,1030.97,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA


### Conciliación